<a href="https://colab.research.google.com/github/Sankalp-sa/Applied_linear_algebra_project/blob/main/ALA_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math as Math
from math import sqrt
from random import normalvariate


In [ ]:
'''
Returns transpose of A

'''


def transpose(A):
    rowA = len(A)
    colA = len(A[0])
    A_T = [[A[j][i] for j in range(rowA)] for i in range(colA)]
    return A_T


'''
Returns multiplication of two rectangular matrices each having more than 2 rows and columns

'''


def dot(A, B):
    rowA = len(A)
    rowB = len(B)
    colB = len(B[0])
    mul = [[0 for i in range(colB)] for j in range(rowA)]
    for i in range(rowA):
        for j in range(colB):
            for k in range(rowB):
                mul[i][j] += A[i][k] * B[k][j]
    return mul


''''
Returns multiplication of one rectangular matrix and 1 dimensional vector (Column Matrix) 

'''


def dotnd_1d(A, v):
    rowA = len(A)
    mul = []
    for i in range(rowA):
        mul.append(dot1d_1d(A[i], v))
    return mul


'''
Returns multiplication of two 1 dimensional vectors

'''


def dot1d_1d(v1, v2):
    rowv1 = len(v1)
    mul = []
    for i in range(rowv1):
        mul.append(v1[i] * v2[i])
    return sum(mul)


'''
Return the outerproduct of two 1 dimensional vectors

'''


def outerproduct(v1, v2):
    res = v1.reshape(-1, 1) * v2
    return res


'''
Returns the norm of a 1 dimensional vector

'''


def norm1d(v):
    return sqrt(sum(i ** 2 for i in v))


'''
Returns unit vector having each entry random normal distribution floating number having mean = 0 and sigma = 1

'''


def randomUnitVector(n):
    unnormalizedVector = [normalvariate(0, 1) for i in range(n)]
    unnormalizedVectorNorm = sqrt(sum(v * v for v in unnormalizedVector))
    return [v / unnormalizedVectorNorm for v in unnormalizedVector]


'''
Performs one dimensional SVD and returns a vector

'''


def svd_1d(A):
    row, col = A.shape
    v = randomUnitVector(min(row, col))
    currentV = v
    previousV = None
    A_T = transpose(A)
    if row > col:
        B = dot(A_T, A)
    else:
        B = dot(A, A_T)

    while (1):
        previousV = currentV
        currentV = dotnd_1d(B, previousV)
        n = np.float64(norm1d(currentV))
        currentV = currentV / n

        if abs(dot1d_1d(currentV, previousV)) > 1:  
            return currentV


'''
Performs SVD using Power method and returns matrix U, Sigma and transpose of V

Here, k is the number largest of singular values you want to compute of input matrix
                if k is None than it computes:
                    matrix U of dimension row x min(row, col)
                    matrix V of dimension min(row, col) x col
                    array Sigma of dimension 1 x min(row, col)

                else:
                    matrix U of dimension row x k
                    matrix V of dimension k x col
                    array Sigma of dimension 1 x k

'''


def svd(A, k=None):
    A = np.array(A, dtype=float)
    row, col = A.shape
    svdCurrent = []
    if k is None:
        k = min(row, col)

    for i in range(k):
        svd1dMatrix = A.copy()

        for u, singularValue, v in svdCurrent[:i]:
            svd1dMatrix -= singularValue * outerproduct(u, v)

        if row > col:
            v = svd_1d(svd1dMatrix)
            u = np.dot(A, v)
            sigma = np.float64(norm1d(u))
            u = u / sigma
        else:
            u = svd_1d(svd1dMatrix)
            v = dotnd_1d(transpose(A), u)
            sigma = np.float64(norm1d(v))
            v = v / sigma

        svdCurrent.append((u, sigma, v))

    us, singularValues, vs = [np.array(x) for x in zip(*svdCurrent)]
    us_T = np.array(transpose(us))
    return us_T, singularValues, vs


In [ ]:
# assign and open image
url = "https://i.picsum.photos/id/494/200/300.jpg?hmac=YdLwRbrTAzFXaAJcsj854mgNuS5jqYM8bcjCzSrSDRM"
response = requests.get(url, stream=True)

with open('image.png', 'wb') as f:
    f.write(response.content)
 
img = cv2.imread('image.png')
 
# Converting the image into gray scale for faster computation.
gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Calculating the SVD

u, s, v = svd(gray_image,None)

# inspect shapes of the matrice
print(f'u.shape:{u.shape},s.shape:{s.shape},v.shape:{v.shape}')

comps = [200, 1, 5, 10, 15, 20]
plt.figure(figsize=(12, 6))
 
for i in range(len(comps)):
    low_rank = u[:, :comps[i]] @ np.diag(s[:comps[i]]) @ v[:comps[i], :]
     
    if(i == 0):
      plt.subplot(2, 3, i+1),
      plt.imshow(low_rank, cmap='gray'),
      plt.title(f'Actual Image reconstructed \n using all singular values')
      
    else:
      plt.subplot(2, 3, i+1),
      plt.imshow(low_rank, cmap='gray'),
      plt.title(f'{comps[i]} singular values') 



    